## Part 1: Preprocessing

In [362]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [363]:
# Determine the number of unique values in each column.
unique_values = attrition_df.nunique()
print(unique_values)

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64


In [364]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
print(y_df.head())


  Attrition              Department
0       Yes                   Sales
1        No  Research & Development
2       Yes  Research & Development
3        No  Research & Development
4        No  Research & Development


In [365]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'BusinessTravel', 'DistanceFromHome', 'Education', 
                    'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 
                    'JobLevel', 'MaritalStatus', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
print(X_df.dtypes)


Age                         int64
BusinessTravel             object
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
MaritalStatus              object
NumCompaniesWorked          int64
dtype: object


In [366]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Optional: Verify the sizes of the training and testing sets
print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (1176, 10)
Testing set size: (294, 10)


In [367]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

X_train, X_test = X_train.align(X_test, join='inner', axis=1)  # Ensuring same columns in both sets

print("Transformed feature names:", X_train.columns)


        

Transformed feature names: Index(['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'NumCompaniesWorked',
       'BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'BusinessTravel_Travel_Rarely', 'MaritalStatus_Divorced',
       'MaritalStatus_Married', 'MaritalStatus_Single'],
      dtype='object')


In [368]:
# Create a StandardScale

scale = StandardScaler()
# Fit the StandardScaler to the training data
x_train_scaled = scale.fit_transform(X_train)

# Scale the training and testing data
x_test_scaled = scale.transform(X_test)
        
print("Scaled training data shape:", x_train_scaled.shape)
print("Scaled testing data shape:", x_test_scaled.shape)

Scaled training data shape: (1176, 14)
Scaled testing data shape: (294, 14)


In [369]:
# Import OneHotEncoder from sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder

# Create an instance of OneHotEncoder for the Department column
encoder_department = OneHotEncoder()   

# Fit the encoder to the training data
encoder_department.fit(y_train[['Department']])  

# Create two new variables by applying the encoder to the training and testing data
# Convert the sparse matrix to a dense array
y_train_department_encoded = encoder_department.transform(y_train[['Department']]).toarray()
y_test_department_encoded = encoder_department.transform(y_test[['Department']]).toarray()

print("Encoded training data shape:", y_train_department_encoded.shape)
print("Encoded testing data shape:", y_test_department_encoded.shape)

Encoded training data shape: (1176, 3)
Encoded testing data shape: (294, 3)


In [370]:
# Create a OneHotEncoder for the Attrition column
encoder_attrition = OneHotEncoder()

# Fit the encoder to the training data
encoder_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_attrition_encoded = encoder_attrition.transform(y_train[['Attrition']]).toarray()
y_test_attrition_encoded = encoder_attrition.transform(y_test[['Attrition']]).toarray()

print("Encoded training data shape:", y_train_attrition_encoded.shape)
print("Encoded testing data shape:", y_test_attrition_encoded.shape)


Encoded training data shape: (1176, 2)
Encoded testing data shape: (294, 2)


## Create, Compile, and Train the Model

In [371]:
# Find the number of columns in the X training data
num_columns = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer1 = layers.Dense(128, activation='relu')(input_layer)
shared_layer2 = layers.Dense(64, activation='relu')(shared_layer1)

In [372]:
# Create a branch for Department
# with a hidden layer and an output layer
    
# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(y_train_department_encoded.shape[1], activation='softmax', name='department_output')(department_hidden)




In [373]:
# Create a branch for Attrition
# with a hidden layer and an output layer


# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_hidden)


In [374]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])
# Compile the model
model.compile(optimizer='adam', 
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'},
              loss_weights={'department_output': 1.0, 'attrition_output': 1.0})
# Summarize the model
model.summary()

Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_67 (Dense)    │ (None, 128)       │      1,920 │ input_layer_13[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_68 (Dense)    │ (None, 64)        │      8,256 │ dense_67[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_69 (Dense)    │ (None, 32)        │      2,080 │ dense_68[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_70 (Dense)    │ (None, 32)        │      2,080 │ dense_68[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_69[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_70[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,501 (56.64 KB)

 Trainable params: 14,501 (56.64 KB)

 Non-trainable params: 0 (0.00 B)

In [375]:
# Train the model
model.fit(x_train_scaled, {'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded}, epochs=100, batch_size=10, verbose=1)

Epoch 1/100


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 918us/step - attrition_output_accuracy: 0.8085 - department_output_accuracy: 0.5177 - loss: 1.4893
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - attrition_output_accuracy: 0.8316 - department_output_accuracy: 0.6638 - loss: 1.1459
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - attrition_output_accuracy: 0.8445 - department_output_accuracy: 0.6571 - loss: 1.0993
Epoch 4/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - attrition_output_accuracy: 0.8477 - department_output_accuracy: 0.6442 - loss: 1.1022
Epoch 5/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - attrition_output_accuracy: 0.8501 - department_output_accuracy: 0.6610 - loss: 1.0587
Epoch 6/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - attrition_output_accuracy: 0.8634 - department_output_accuracy: 0.7024 - loss: 1.0086
Epoch 7/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - attrition_output_accuracy: 0.8463 - department_output_accuracy: 0.6909 - loss: 1.0148
Epoch 8/100

In [376]:
# Evaluate the model with the testing data
results = model.evaluate(x_test_scaled, 
                         {'department_output': y_test_department_encoded, 
                          'attrition_output': y_test_attrition_encoded}, 
                         verbose=1)



10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - attrition_output_accuracy: 0.7981 - department_output_accuracy: 0.5537 - loss: 5.6550


In [377]:
# Print the accuracy for both department and attrition
pred_categories = ['Department', 'Attrition']
for i, cat in enumerate(pred_categories):
    print(f"{cat} accuracy: {results[i]}")



Department accuracy: 5.261289596557617
Attrition accuracy: 0.8163265585899353


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Evaluate the distribution of the target variables**. If an imbalance is observed, accuracy may not serve as the most effective metric. Other metrics such as precision, recall, or the F1 score might be more suitable.

2. For the 'Department' variable, a softmax function** would be appropriate as it is a multi-class problem. For the 'Attrition' variable, a sigmoid function** would be suitable as it is a binary classification problem.

3. Experiment with various network architectures**, including adding or removing layers, or incorporating different types of layers (for instance, dropout layers for regularization). Additionally,hyperparameters such as the learning rate and batch size should be tuned** for optimal performance.

Consider employing more advanced techniques like feature engineering** or using more sophisticated encoders for handling categorical data to improve the model's performance.

